Modelo de AI para prever qual será o score_credito de um cliente.
Será usado modelo de aprendizado supervisionado com a biblioteca scikit-learn.

In [1]:
#Importando as bibliotecas que serão usadas para prever a váriavel target.

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [2]:
#Importando a base de dados
df = pd.read_csv(r'\datasets\clientes.csv')

#Limpando e tratando os dados
#df.isna().sum() #nenhum valor nulo nas colunas
#df.info()

#removendo qualquer espaço a mais no inicio e final das colunas que são texto
cols = ['profissao', 'mix_credito', 'comportamento_pagamento']

for col in cols:
    df[col] = df[col].str.strip()

Preparando a base de dados.
Tratando colunas que não estão como números, a inteligencia artificial trabalha somente com números.
As colunas que não são números são: 
profissao
mix_credito
comportamento_pagamento
score_credito

A coluna score_credito não precisará ser tratada pois ela é nosso target e não entrará como variável de treinamento do modelo
 


In [3]:
#Criando o codificador que transforma os dados de texto para numero

encoder = LabelEncoder()

df['profissao'] = encoder.fit_transform(df['profissao'])
df['mix_credito'] = encoder.fit_transform(df['mix_credito'])
df['comportamento_pagamento'] = encoder.fit_transform(df['comportamento_pagamento'])

In [4]:
#help(train_test_split)

In [4]:
#Separando as colunas que serão usadas para treinar o modelo, serão armazenadas em X. y é o obejto com a variável Target.

X = df.drop(['score_credito', 'id_cliente'], axis = 1).values #Removi também a coluna id_cliente porque ela não deve ser levada em consideração no aprendizado da AI
y = df['score_credito'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

Criando instância do modelo que fará a previsão. Faremos a comparação entre três modelos, RandomForestClassifier, KNeighborsClassifier e LinearRegression

In [5]:
#Criando instância do modelo

forest = RandomForestClassifier()
knn = KNeighborsClassifier(n_neighbors = 5)
#linear = LinearRegression()

#Treinando os modelos. Ao se treinar os modelos devemos usar as variáveis de treino. Uma analogia para lembrar: para ficar fitness temos que treinar!
forest.fit(X_train, y_train)
knn.fit(X_train, y_train)
#linear.fit(X_train, y_train)

#Fazendo a previsão dos dados de teste que separamos. Usamos sempre os dados que são as caracteristicas do modelo ao fazer o .predict()
for_prediction = forest.predict(X_test)
knn_prediction = knn.predict(X_test)
#linear.predict(X_test)

Escolhendo qual o melhor modelo, calculando a Accuracy de cada modelo

In [6]:
#Printando qual modelo teve melhor resultado de Accuracy. Neste teste vemos qual modelo acertou mais vezes as previsões

for_accuracy = accuracy_score(y_test, for_prediction)
knn_accuracy = accuracy_score(y_test, knn_prediction)

print('Accuracy do modelo RandomForest: {}'.format(np.round(for_accuracy * 100, 2)), '%')
print('Accuracy do modelo KNeighbors: {}'.format(np.round(knn_accuracy * 100, 2)),'%')

Accuracy do modelo RandomForest: 82.45 %
Accuracy do modelo KNeighbors: 73.75 %


Chegamos a conclusão que o modelo RandomForestClassifier conseguiu prever corretamente mais vezes que o modelo KNeighborsClassifier.

Utilizando o modelo RandomForestClassifier em um cenário real:

In [11]:
#Importando a nova base com novos clientes
n_df = pd.read_csv(r'datasets\novos_clientes.csv')

#Fazendo o tratamento da base de dados. Transformando as colunas de texto em numericas
n_df['profissao'] = encoder.fit_transform(n_df['profissao'])
n_df['mix_credito'] = encoder.fit_transform(n_df['mix_credito'])
n_df['comportamento_pagamento'] = encoder.fit_transform(n_df['comportamento_pagamento'])

Index(['mes', 'idade', 'profissao', 'salario_anual', 'num_contas',
       'num_cartoes', 'juros_emprestimo', 'num_emprestimos', 'dias_atraso',
       'num_pagamentos_atrasados', 'num_verificacoes_credito', 'mix_credito',
       'divida_total', 'taxa_uso_credito', 'idade_historico_credito',
       'investimento_mensal', 'comportamento_pagamento', 'saldo_final_mes',
       'emprestimo_carro', 'emprestimo_casa', 'emprestimo_pessoal',
       'emprestimo_credito', 'emprestimo_estudantil'],
      dtype='object')


In [8]:
#Como o modelo de previsão já está pronto, não precisamos fazer um novo, basta fazer a previsão nesta nova base
previsao = forest.predict(n_df)

c:\Users\Matheus\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [12]:
n_df['score_predict'] = previsao
n_df[['idade', 'profissao', 'salario_anual', 'score_predict']]

,idade,profissao,salario_anual,score_predict
0,31.0,1,19300.340,Poor
1,32.0,0,12600.445,Standard
2,48.0,1,20787.690,Standard
